In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

from src.io_handler import RustMonteCarloData, perform_rust_computation, write_parameter_file

## Perform Monte-Carlo calculation:

Overview!!

In [ ]:
try: 
    os.mkdir("results/overview")
except Exception as e:
    print(f"Cannot create dir: {e}")

In [ ]:
temperatures         = np.arange(1, 3, 0.05)
lengths              = [8, 16, 32]#, 64, 128]
thermalisation_steps = dict()
measurement_steps    = dict()

param_files = dict()
out_files   = dict()

for L in lengths:
    if L <= 64:
        thermalisation_steps[L] = np.uint(5e5)
        measurement_steps[L]    = np.uint(5e5)
    else:
        thermalisation_steps[L] = np.uint(1e5)
        measurement_steps[L]    = np.uint(1e5)
            
    param_files[L] = f"results/overview/parameter_{L}x{L}.txt"
    out_files[L]   = f"results/overview/out_{L}x{L}.txt"
        
    write_parameter_file(param_files[L], out_files[L], L, L, thermalisation_steps[L], measurement_steps[L], temperatures)

In [ ]:
for L in lengths:
    # if L != 128:
    perform_rust_computation(param_files[L])

Load & plot from file:

In [ ]:
my_lengths      = [8, 16, 32]#, 64, 128]
results: dict[int, RustMonteCarloData] = dict()

for L in my_lengths:
    results[L] = RustMonteCarloData(f"results/overview/out_{L}x{L}.txt")

    print(f"elapsed time for size {L}x{L}: {results[L].elapsed_time/60:.3}min ({results[L].elapsed_time}s)")

print("Assert: all use the same temperatures.")
for res1 in results.values():
    for res2 in results.values():
        if res1 is not res2:
            have_same_temp = np.all(res1.temperatures == res2.temperatures)
            assert(have_same_temp)

temps = results[lengths[0]].temperatures

### Plot!

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,10))

fig.suptitle(f"2D Ising in Rust", y=0.95, fontsize=20)
for (L, result) in results.items():

    ax1.scatter(temps, result.energy_density, s=15, marker= 'x', label=f"{L}x{L}")
    ax1.plot(temps, result.energy_density)#, label="Rust" )
    ax1.set_title(f"energy density vs temp, L={L}")
    ax1.legend(loc="upper left",fontsize=14)

    ax2.scatter(temps, result.magnetisation, s=15, marker= 'x', label=f"{L}x{L}")
    ax2.plot(temps, result.magnetisation)#, label="Rust")
    ax2.set_title(f"magnetization density vs temp, L={L}")
    ax2.legend(loc="upper right",fontsize=14)

    ax3.scatter(temps, result.specific_heat, s=15, marker= 'x', label=f"{L}x{L}")
    ax3.plot(temps, result.specific_heat)#, label="Rust")
    ax3.set_title(f"specific heat/spin vs temp, L={L}")
    ax3.legend(loc="upper left",fontsize=14)

    ax4.scatter(temps, result.mag_susceptibility, s=15, marker= 'x', label=f"{L}x{L}")
    ax4.semilogy(temps, result.mag_susceptibility)
    ax4.set_title(f"magnetic susceptibility/spin vs temp, L={L}")
    ax4.legend(loc="upper left",fontsize=14)